In [2]:
import pandas as pd
import json
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
)
from typing import List, Dict
import seaborn as sns
import matplotlib.pyplot as plt
from emoji import demojize
from nltk.tokenize import TweetTokenizer

df_davinci_v01 = pd.read_csv('../data/weak_labeled_data/davinci_elaboration_first_v04_name_weak_labels_all.csv')
df_oa_v03 = pd.read_csv('../data/weak_labeled_data/oa_classification_only_v03_name_weak_labels_all.csv')
df_oa_v02 = pd.read_csv('../data/weak_labeled_data/oa_with_3_random_examples_classification_only_name_weak_labels_all.csv')
df_oa_v01 = pd.read_csv('../data/weak_labeled_data/oa_with_rules_classification_only_name_weak_labels_all.csv')
df_vicuna_v01_full = pd.read_csv('../data/weak_labeled_data/vicuna_with_rules_classification_only_name_full_weak_labels_all.csv')
df_vicuna_v01 = pd.read_csv('../data/weak_labeled_data/vicuna_with_rules_classification_only_name_weak_labels_all.csv')

/tmp/ipykernel_78679/4294735786.py:25: DtypeWarning: Columns (0,1,9,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vicuna_v01_full = pd.read_csv('../data/weak_labeled_data/vicuna_with_rules_classification_only_name_full_weak_labels_all.csv')


In [3]:
df_vicuna_v01

,index,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,...,urls,user_mentions,poll_choices,Conspiracy Theory_pred,Education_pred,Environment_pred,Labor/Employment_pred,Religion_pred,Science/Technology_pred,annotations
0,1254904,825252864771567617,z1HuUl+9Gxe6nIzglABuIKq1xjSlhgcvfDt9ldrk7Nc=,z1HuUl+9Gxe6nIzglABuIKq1xjSlhgcvfDt9ldrk7Nc=,z1HuUl+9Gxe6nIzglABuIKq1xjSlhgcvfDt9ldrk7Nc=,Estados Unidos,FOLLOW AND DISCOVER ALL THE LAST MINUTE. TOP U...,NaN,2282.0,5190,...,['http://viid.me/qgM71x'],[],NaN,0 (Not about Conspiracy Theory),0\n\nThe tweet is not directly,0 (Not related to environment)\n\n,0 (Not related)\n\nExplan,0 (Not related to religion)\n\n,0 (Not about Science/Technology,[]
1,1082628,816229906983964672,EnJOMZA4bIqKIG7CS5cI7OPKgxg6HAIo4+AwnWjN44=,EnJOMZA4bIqKIG7CS5cI7OPKgxg6HAIo4+AwnWjN44=,EnJOMZA4bIqKIG7CS5cI7OPKgxg6HAIo4+AwnWjN44=,Estados Unidos,All information of america and more.. TOP USA ...,NaN,2072.0,2243,...,['http://viid.me/qusfjB'],[],NaN,0 (Not related)\n\nExplan,0\n\nReasoning: The twe,0 (Not related)\n\nExplan,0 (not related)\n\nExplan,0 (Not related)\n\nExplan,0 (Not about Science/Technology,[]
2,1293885,903817338709008386,wwec6zN1bIVqojfFvFx9D3TGpTTqerWyIcvmP0tIDaE=,wwec6zN1bIVqojfFvFx9D3TGpTTqerWyIcvmP0tIDaE=,wwec6zN1bIVqojfFvFx9D3TGpTTqerWyIcvmP0tIDaE=,Estados Unidos,But he made too many enemies Of the people who...,https://t.co/4srF9ALSJp,1893.0,84,...,['http://zpr.io/PQUJA'],[],NaN,0 (Not about Conspiracy Theory),0\n\nThe tweet is not related,0 (Not related)\n\nExplan,0 (Not related)\n\nExplan,0 (Not related)\n\nExplan,0 (Not about Science/Technology,[]
3,187033,1063439406361731073,xJ57jjSHWvX9mAMmhv7fVaVzxe13bBfCZuGZaBNucL0=,xJ57jjSHWvX9mAMmhv7fVaVzxe13bBfCZuGZaBNucL0=,xJ57jjSHWvX9mAMmhv7fVaVzxe13bBfCZuGZaBNucL0=,"Kampala, Uganda",this page for teamshirumatic updates and the a...,https://t.co/oT1yUpwAsC,4469.0,713,...,[],"['597473756', '8789142', '359814192', '1608515...",NaN,0 (Not about Conspiracy Theory),0\n\nThe tweet is not related,0 (Not related to environment)\n\n,0 (Not about Labor/Employment),0 (Not related to religion)\n\n,0 (Not about Science/Technology,[]
4,281944,1186641059843063808,830195087078518785,ssebunya shafique,ssebunyashaf,Uganda,economics politics music and sports\n@arsenal ...,https://t.co/4lf4iRcXJT,15717.0,4232,...,[],"['LxIZFKrgNsaCLceCiBg56S2BLCm2FvPNhlC2Hizos=',...",NaN,0\n\nExplanation: This twe,0\n\nThe tweet is not directly,0\n\nReason: The tweet,0\n\nThe tweet does not appear,0\n\nExplanation: The twe,0\n\nThe tweet is not directly,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,104856,644873385747578880,4IESrF58WF9LX1a3RySuP3WebiusoHnD+ZnAr1wmWd0=,4IESrF58WF9LX1a3RySuP3WebiusoHnD+ZnAr1wmWd0=,4IESrF58WF9LX1a3RySuP3WebiusoHnD+ZnAr1wmWd0=,Uganda,Supporting the campaign & re-election of @Kagu...,NaN,1925.0,735,...,[],[],NaN,0\n\nThe tweet is related to,0\nReasoning: The tweet,0\n\nReason: The tweet,0\n\nExplanation: The twe,0\n\nExplanation: The twe,0\nReason: The tweet does,[]
1996,836894,873538330679549952,zjM1VynckXRrGwh4gmb384xY3JQpuVwZE7EqEbpbQ7c=,zjM1VynckXRrGwh4gmb384xY3JQpuVwZE7EqEbpbQ7c=,zjM1VynckXRrGwh4gmb384xY3JQpuVwZE7EqEbpbQ7c=,Estados Unidos,We take the best information fast and timely p...,NaN,4044.0,3185,...,['http://clkmein.com/qKqgm2'],[],NaN,0 (Not about Conspiracy Theory),0\n\nExplanation: The twe,0 (Not related to environment)\n\n,0 (Not related)\n\nExplan,0 (Not related to religion)\n\n,0 (Not about Science/Technology,[]
1997,641578,826016295174836226,zYH8Owk82nvA7h1yflkRduYaxZUIFlvrBnJjoMvH6cI=,zYH8Owk82nvA7h1yflkRduYaxZUIFlvrBnJjoMvH6cI=,zYH8Owk82nvA7h1yflkRduYaxZUIFlvrBnJjoMvH6cI=,NaN,"If you want Twee that do not bore you, then fo...",NaN,2.0,24,...,['http://viid.me/qhKinb'],[],NaN,0 (Not a Conspiracy Theory),0\n\nReasoning: The twe,0\n\nReason: The tweet,0 (Not related)\n\nThe twe,0 (Not related to re